In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pygimli as pg
import pygimli.meshtools as mt
import pygimli.physics.ert as ert

In [2]:
number_of_tests = 50
rho_spread_factor = 1.5
rho_max = 150
layers_min = 1
layers_max = 5
min_depth = 1
max_depth = 25

world_boundary_v = [-10 * max_depth, 0]  # [right, left border] relatively to the middle
world_boundary_h = [10 * max_depth, -4 * max_depth]  # [top, bottom border]


In [3]:
layers_pos = [-8, -18]
resistivity_map = [[1, 20], [2, 50], [3, 150]]

In [5]:


test_results = {}

# INPUT MODEL - SUBSURFACE START #
world = mt.createWorld(start=world_boundary_v, end=world_boundary_h,
                       layers=layers_pos)  # ,
# marker=np.linspace(1, tests_horizontal['layer_n']['hor_1'],
#                  tests_horizontal['layer_n']['hor_1']))

geometry = world  # +block

measurement_scheme = ert.createERTData(elecs=np.linspace(start=-45, stop=45, num=91), schemeName='dd')
for electrode in measurement_scheme.sensors():
    geometry.createNode(electrode)
    geometry.createNode(electrode - [0, 0.1])  # What does it do?

mesh = mt.createMesh(geometry, quality=34)  # , area=2)#

 # [0]

input_model = pg.solver.parseMapToCellArray(resistivity_map, mesh)  # rename to input_mesh

# INPUT MODEL - SUBSURFACE MODEL END ###

# SIMULATE ERT MEASUREMENT - START ###
data = ert.simulate(mesh, scheme=measurement_scheme, res=resistivity_map, noiseLevel=1, noiseAbs=1e-6, seed=1337)
data.remove(data['rhoa'] < 0)
# SIMULATE ERT MEASUREMENT - END ###

ert_manager = ert.ERTManager(sr=False, useBert=True, verbose=True, debug=False)

# RUN INVERSION #
k0 = pg.physics.ert.createGeometricFactors(data)
model_inverted = ert_manager.invert(data=data, lam=20, paraDX=0.25, paraMaxCellSize=5, paraDepth=max_depth, quality=34,
                                    zPower=0.4)
result = ert_manager.inv.model
result_array = result.array()

input_model2 = pg.interpolate(srcMesh=mesh, inVec=input_model, destPos=ert_manager.paraDomain.cellCenters())

input_model2_array = input_model2.array()

experiment_results = pd.DataFrame(data={'X': ert_manager.paraDomain.cellCenters().array()[:, 0],
                                        'Y': ert_manager.paraDomain.cellCenters().array()[:, 1],
                                        'Z': ert_manager.paraDomain.cellCenters().array()[:, 2],
                                        'INM': input_model2_array,
                                        'RES': result_array})#,
                                        #'CLASS': input_class})


#experiment_results.to_csv('results/results/'+test_name+'.csv')

21/01/21 - 12:08:55 - pyGIMLi - INFO - Cache /home/felikskrno/anaconda3/envs/MasterEnv/lib/python3.7/site-packages/pygimli/physics/ert/ert.py:createGeometricFactors restored (0.0s x 3): /home/felikskrno/.var/app/com.jetbrains.PyCharm-Professional/config/pygimli/15058862444113807336
21/01/21 - 12:08:55 - pyGIMLi - INFO - Found 2 regions.
21/01/21 - 12:08:55 - pyGIMLi - INFO - Region with smallest marker set to background (marker=1)
21/01/21 - 12:08:55 - pyGIMLi - INFO - Creating forward mesh from region infos.
21/01/21 - 12:08:55 - Core - WARNING - Region Nr: 1  is background and should not get a model transformation.
21/01/21 - 12:08:55 - Core - WARNING - Region Nr: 1  is background and should not get a model control.
21/01/21 - 12:08:55 - pyGIMLi - INFO - Creating refined mesh (H2) to solve forward task.
21/01/21 - 12:08:55 - pyGIMLi - INFO - Starting inversion.
21/01/21 - 12:08:55 - pyGIMLi - INFO - Set default startmodel to median(data values)=26.606888203853856
21/01/21 - 12:08:55 

relativeError set to a value > 0.5 .. assuming this is a percentage Error level dividing them by 100
Data error estimate (min:max)  0.010000943083472701 : 0.0459008826982054
<class 'pygimli.physics.ert.ert.ERTManager'>.applyMesh(methodManager.py:647) : Mesh: Nodes: 1745 Cells: 3160 Boundaries: 4904
fop: <pygimli.physics.ert.ert.ERTModelling object at 0x7f1219d7d530>
Data transformation: <pygimli.core._pygimli_.RTransLogLU object at 0x7f1219d7d3f0>
Model transformation: <pygimli.core._pygimli_.RTransLog object at 0x7f1219d7d4b0>
min/max (data): 18.96/63.91
min/max (error): 1.0%/4.59%
min/max (start model): 26.61/26.61
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
inv.iter 2 ... chi² = 157.55 (dPhi = 23.86%) lam: 20.0
--------------------------------------------------------------------------------
inv.iter 3 ... chi² = 157.64 (dPhi = -0.06%) lam: 20.0
----------------------

In [22]:
# Caluculate the Jacobian
jac_mat = ert.ERTModelling()
jac_mat.setData(measurement_scheme)
jac_mat.setMesh(ert_manager.paraDomain)

jac_model = np.ones(ert_manager.paraDomain.cellCount())
jac_mat.createJacobian(jac_model)

21/01/21 - 12:32:41 - Core - INFO - More than 50 regions so we assume singles only regions.
21/01/21 - 12:32:41 - Core - INFO - Applying *:* interregion constraints.
21/01/21 - 12:32:42 - pyGIMLi - INFO - Found 2373 regions.
21/01/21 - 12:32:42 - pyGIMLi - INFO - Region with smallest marker set to background (marker=0)
21/01/21 - 12:32:42 - pyGIMLi - INFO - Creating forward mesh from region infos.
21/01/21 - 12:32:42 - Core - WARNING - Region Nr: 0  is background and should not get a model transformation.
21/01/21 - 12:32:42 - Core - WARNING - Region Nr: 0  is background and should not get a model control.
21/01/21 - 12:32:42 - pyGIMLi - INFO - Creating refined mesh (H2) to solve forward task.


In [23]:
print(jac_mat.jacobian())

RMatrix: 3916 x 2372


In [24]:
for i, sens in enumerate(jac_mat.jacobian()):
    #print(i)
    print(sens)
    print(ert_manager.paraDomain.cellSizes())
    print(sens/ert_manager.paraDomain.cellSizes())
    normsens = pg.utils.logDropTol(sens/ert_manager.paraDomain.cellSizes(), 8e-4)
    normsens /= np.max(normsens)
    pg.show(mesh, normsens, cMap="RdGy_r", orientation="vertical",
            label="Normalized\nsensitivity", nLevs=3, cMin=-1, cMax=1)

2372 [3.023983379103861e-12,...,1.234553572748536e-10]
2373 [0.5457237210451623,...,2.049497903414853]


RuntimeError: /home/wagner/miniconda3/conda-bld/pygimli_1589132014332/work/gimli/gimli/core/src/vector.h: 671		GIMLI::Vector<ValueType>& GIMLI::Vector<ValueType>::operator/=(const GIMLI::Vector<ValueType>&) [with ValueType = double]  2372 != 2373

In [25]:
# Caluculate the Jacobian with mesh
jac_mat = ert.ERTModelling()
jac_mat.setData(measurement_scheme)
jac_mat.setMesh(mesh)

jac_model = np.ones(mesh.cellCount())
jac_mat.createJacobian(jac_model)

21/01/21 - 12:46:29 - pyGIMLi - INFO - Found 3 regions.
21/01/21 - 12:46:29 - pyGIMLi - INFO - Region with smallest marker set to background (marker=1)
21/01/21 - 12:46:29 - pyGIMLi - INFO - Creating forward mesh from region infos.
21/01/21 - 12:46:29 - Core - WARNING - Region Nr: 1  is background and should not get a model transformation.
21/01/21 - 12:46:29 - Core - WARNING - Region Nr: 1  is background and should not get a model control.
21/01/21 - 12:46:29 - pyGIMLi - INFO - Creating refined mesh (H2) to solve forward task.


In [26]:
print(jac_mat.jacobian())

RMatrix: 3916 x 991


In [27]:
for i, sens in enumerate(jac_mat.jacobian()):
    #print(i)
    print(sens)
    print(mesh.cellSizes())
    print(sens/mesh.cellSizes())
    normsens = pg.utils.logDropTol(sens/mesh.cellSizes(), 8e-4)
    normsens /= np.max(normsens)
    pg.show(mesh, normsens, cMap="RdGy_r", orientation="vertical",
            label="Normalized\nsensitivity", nLevs=3, cMin=-1, cMax=1)


991 [4.825647623638317e-09,...,9.402004956049018e-11]
4729 [4.545606125185435,...,294.73664991790736]


RuntimeError: /home/wagner/miniconda3/conda-bld/pygimli_1589132014332/work/gimli/gimli/core/src/vector.h: 671		GIMLI::Vector<ValueType>& GIMLI::Vector<ValueType>::operator/=(const GIMLI::Vector<ValueType>&) [with ValueType = double]  991 != 4729

In [28]:
# Caluculate the Jacobian with ert_manager.paraDomain
jac_mat = ert.ERTModelling()
jac_mat.setData(measurement_scheme)
jac_mat.setMesh(ert_manager.paraDomain)

jac_model = np.ones(ert_manager.paraDomain.cellCount())
jac_mat.createJacobian(jac_model)


21/01/21 - 12:48:15 - Core - INFO - More than 50 regions so we assume singles only regions.
21/01/21 - 12:48:15 - Core - INFO - Applying *:* interregion constraints.
21/01/21 - 12:48:16 - pyGIMLi - INFO - Found 2373 regions.
21/01/21 - 12:48:16 - pyGIMLi - INFO - Region with smallest marker set to background (marker=0)
21/01/21 - 12:48:16 - pyGIMLi - INFO - Creating forward mesh from region infos.
21/01/21 - 12:48:16 - Core - WARNING - Region Nr: 0  is background and should not get a model transformation.
21/01/21 - 12:48:16 - Core - WARNING - Region Nr: 0  is background and should not get a model control.
21/01/21 - 12:48:16 - pyGIMLi - INFO - Creating refined mesh (H2) to solve forward task.


In [29]:
print(jac_mat.jacobian())

RMatrix: 3916 x 2372


In [30]:
for i, sens in enumerate(jac_mat.jacobian()):
    #print(i)
    print(sens)
    print(ert_manager.paraDomain.cellSizes())
    print(sens/ert_manager.paraDomain.cellSizes())
    normsens = pg.utils.logDropTol(sens/ert_manager.paraDomain.cellSizes(), 8e-4)
    normsens /= np.max(normsens)
    pg.show(ert_manager.paraDomain, normsens, cMap="RdGy_r", orientation="vertical",
            label="Normalized\nsensitivity", nLevs=3, cMin=-1, cMax=1)

2372 [3.023983379103861e-12,...,1.234553572748536e-10]
2373 [0.5457237210451623,...,2.049497903414853]


RuntimeError: /home/wagner/miniconda3/conda-bld/pygimli_1589132014332/work/gimli/gimli/core/src/vector.h: 671		GIMLI::Vector<ValueType>& GIMLI::Vector<ValueType>::operator/=(const GIMLI::Vector<ValueType>&) [with ValueType = double]  2372 != 2373